In [3]:
pip install torch

Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Ilya Jeff\AppData\Local\Programs\Python\Python39\python.exe -m pip install --upgrade pip' command.


In [1]:
import seaborn as sns
import glob
import torch
import os
import numpy as np

import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
sns.set(style="whitegrid")
mpl.rcParams['agg.path.chunksize'] = 10000

#from matplotlib import rc
#rc('text', usetex=True)
mpl.rcParams['axes.titlesize'] = 'xx-large'
mpl.rcParams['axes.labelsize'] = 'xx-large'


In [5]:
rerun_rounds = ['round1', 'round2', 'round3']
def get_avg_results(path_string):
  """
  For experiments with multiple random training runs, collects all
  training runs and averages their results
  """
  result_list = []
  for rerun_string in rerun_rounds:
    completed_path = path_string.format(rerun_string)
    if os.path.exists(completed_path):
      result_list.append(float(open(completed_path).read()))
  return sum(result_list)/len(result_list) if len(result_list) > 0 else -1.0
  #print(result_list)
  #return np.std(result_list) if len(result_list) > 0 else -1 #sum(result_list)/len(result_list) if len(result_list) > 0 else -1.0

In [70]:
#a = torch.tensor([get_avg_results('reports/dep_rel-6-768-en-distilbert-layer{}.yaml.results/dev.v_entropy'.format(i)) for i in range(1,7)])
#b = torch.tensor([float(open('reports/dep_rel-6-768-en-distilbert-layer{}.yaml.results/dev.v_entropy'.format(i)).read().strip()) for i in range(1,7)])
#c = get_avg_results('reports/layer0.yaml.results/dev.v_entropy')
#print(a)
#print(b)

#for i in range(1,7):
#   print(i-1)    

In [18]:
def plot_layerwise_zero_baseline(task, model):
    fig, ax = plt.subplots(figsize=(6,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,11)))
    """Makes a plot comparing layerwise bits of info"""
    #change range depending on layers of model
    task_768_singlelayer_ventropy = torch.tensor([get_avg_results('reports/{}-{}-layer{}.yaml.results/dev.v_entropy'.format(task,model,i)) for i in range(1,11)])
    task_768_twolayer_ventropy = torch.tensor([get_avg_results('reports/{}-{}-layer{}-0.yaml.results/dev.v_entropy'.format(task,model,i)) for i in range(1,11)])
    task_768_embedding_layer_ventropy = get_avg_results('reports/{}-{}-layer0.yaml.results/dev.v_entropy'.format(task,model))

    #print(task_768_singlelayer_ventropy)
    #print(task_768_twolayer_ventropy)
    #print(task_768_embedding_layer_ventropy)

    #12 layer
    #x_range = list(range(1,13))

    #6 layer
    #x_range = list(range(1,7))

    #3 layer
    #x_range = list(range(1,4))

    #8 layer
    #x_range = list(range(1,9))

    #10 layer
    x_range = list(range(1,11))

    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_singlelayer_ventropy, label=r'baselined')
    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_twolayer_ventropy, label=r'conditional')

    #plot title
    plt.title('{} {}'.format(model,task), fontsize = 12)

    plt.ylabel('Bits', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    plt.legend()
    plt.tight_layout()

    #save figure in folder
    plt.savefig('graphs/{0}/{0} {1}.png'.format(model,task), dpi=200)

    plt.clf()

In [ ]:
plot_layerwise_zero_baseline('dep_rel','distilbert-b64-L10')
plot_layerwise_zero_baseline('named_entities','distilbert-b64-L10')
plot_layerwise_zero_baseline('ptb_pos','distilbert-b64-L10')
plot_layerwise_zero_baseline('upos','distilbert-b64-L10')
plot_layerwise_zero_baseline('word_sense','distilbert-b64-L10')
plot_layerwise_zero_baseline('sst2','distilbert-b64-L10')

In [19]:
def plot_accuracy(task, model):
    fig, ax = plt.subplots(figsize=(6,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,11)))
    """Makes a plot accuracy"""
    #change range depending on layers of model
    label_accuracy = torch.tensor([get_avg_results('reports/{}-{}-layer{}.yaml.results/dev.label_acc'.format(task,model,i)) for i in range(1,11)])

    #print(label_accuracy)

    #12 layer
    #x_range = list(range(1,13))

    #6 layer
    #x_range = list(range(1,7))

    #3 layer
    #x_range = list(range(1,4))

    #8 layer
    #x_range = list(range(1,9))

    #10 layer
    x_range = list(range(1,11))

    plt.plot(x_range, label_accuracy)

    #plot title
    plt.title('{} {} accuracy'.format(model,task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    plt.tight_layout()

    #save figure in folder
    plt.savefig('graphs/{0}/{0} {1} accuracy.png'.format(model,task), dpi=200)

    plt.clf()

In [ ]:
plot_accuracy('dep_rel','distilbert-b64-L10')
plot_accuracy('named_entities','distilbert-b64-L10')
plot_accuracy('ptb_pos','distilbert-b64-L10')
plot_accuracy('upos','distilbert-b64-L10')
plot_accuracy('word_sense','distilbert-b64-L10')
plot_accuracy('sst2','distilbert-b64-L10')

In [33]:
def plot_accuracy_official_berts(task):
    fig, ax = plt.subplots(figsize=(10,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""
    #bert768
    label_accuracy_bert768 = torch.tensor([get_avg_results('reports/{}-bert768-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #bert-l2-prune30
    label_accuracy_bert_l2_prune30 = torch.tensor([get_avg_results('reports/{}-bert-l2-prune30-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #distilbert-base-uncased
    label_accuracy_distilbert_base_uncased = torch.tensor([get_avg_results('reports/{}-distilbert-base-uncased-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #MiniLMv2-L6-H768-distilled-from-BERT-Base
    label_accuracy_minilmv2 = torch.tensor([get_avg_results('reports/{}-MiniLMv2-L6-H768-distilled-from-BERT-Base-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #print(label_accuracy)

    #12 layer
    x_range1 = list(range(1,13))
    #3 layer
    x_range2 = np.linspace(1,12,3)
    #6 layer
    x_range3 = np.linspace(1,12,6)
    #8 layer
    x_range4 = np.linspace(1,12,8)
    #10 layer
    x_range5 = np.linspace(1,12,10)

    plt.plot(x_range1, label_accuracy_bert768, label=r'bert_768', color='red')
    plt.plot(x_range3, label_accuracy_distilbert_base_uncased, label=r'distilbert-base-uncased', color='green')
    plt.plot(x_range3, label_accuracy_minilmv2, label=r'MiniLMv2', color='blue')
    plt.plot(x_range1, label_accuracy_bert_l2_prune30, label=r'bert-l2-prune30', color='darkorange')

    plt.title('Accuracy of {} for variants of bert scaled for 12 layers'.format(task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize = "small")
    plt.tight_layout()

    plt.savefig('graphs/Accuracies/Accuracies for {} original bert variants scaled to 12 layers.png'.format(task), dpi=200)

    plt.clf()

In [34]:
plot_accuracy_official_berts('dep_rel')
plot_accuracy_official_berts('named_entities')
plot_accuracy_official_berts('ptb_pos')
plot_accuracy_official_berts('upos')
plot_accuracy_official_berts('word_sense')
plot_accuracy_official_berts('sst2')

In [44]:
def plot_conditional_official_bert(task):
    fig, ax = plt.subplots(figsize=(10,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""
    #bert768
    twolayer_bert768 = torch.tensor([get_avg_results('reports/{}-bert768-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded1 = get_avg_results('reports/{}-bert768-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_bert768 = embedded1 - twolayer_bert768

    #bert-l2-prune30
    twolayer_bertl2 = torch.tensor([get_avg_results('reports/{}-bert-l2-prune30-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded2 = get_avg_results('reports/{}-bert-l2-prune30-layer0.yaml.results/dev.v_entropy'.format(task))
    conditiona_l2 = embedded2 - twolayer_bertl2

    #distilbert-base-uncased
    twolayer_distil = torch.tensor([get_avg_results('reports/{}-distilbert-base-uncased-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded3 = get_avg_results('reports/{}-distilbert-base-uncased-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_distil = embedded3 - twolayer_distil

    #MiniLMv2-L6-H768-distilled-from-BERT-Base
    twolayer_minilmv2 = torch.tensor([get_avg_results('reports/{}-MiniLMv2-L6-H768-distilled-from-BERT-Base-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded13 = get_avg_results('reports/{}-MiniLMv2-L6-H768-distilled-from-BERT-Base-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_minilmv2 = embedded13 - twolayer_minilmv2

    #print(label_accuracy)

    #12 layer
    x_range1 = list(range(1,13))
    #3 layer
    x_range2 = np.linspace(1,12,3)
    #6 layer
    x_range3 = np.linspace(1,12,6)
    #8 layer
    x_range4 = np.linspace(1,12,8)
    #10 layer
    x_range5 = np.linspace(1,12,10)

    plt.plot(x_range1, conditional_bert768, label=r'bert_768', color='red')
    plt.plot(x_range3, conditional_distil, label=r'distilbert-base-uncased', color='green')
    plt.plot(x_range3, conditional_minilmv2, label=r'MiniLMv2', color='blue')
    plt.plot(x_range1, conditiona_l2, label=r'bert-l2-prune30', color='darkorange')

    plt.title('V-Entropy of {} for variants of bert scaled for 12 layers'.format(task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize = "small")
    plt.tight_layout()

    plt.savefig('graphs/V-Entropy/V-Entropy for {} original bert variants scaled to 12 layers.png'.format(task), dpi=200)

    plt.clf()

In [45]:
plot_conditional_official_bert('dep_rel')
plot_conditional_official_bert('named_entities')
plot_conditional_official_bert('ptb_pos')
plot_conditional_official_bert('upos')
plot_conditional_official_bert('word_sense')
plot_conditional_official_bert('sst2')

In [79]:
def plot_accuracy_experiments(task):
    fig, ax = plt.subplots(figsize=(10,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""
    #3-768-distilbert
    label_accuracy_3_768 = torch.tensor([get_avg_results('reports/{}-3-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,4)])

    #6-768-distilbert
    label_accuracy_6_768 = torch.tensor([get_avg_results('reports/{}-6-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #6-b128-distilbert
    label_accuracy_6_b128 = torch.tensor([get_avg_results('reports/{}-6-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #6-768-b32-distilbert
    label_accuracy_6_768_b32 = torch.tensor([get_avg_results('reports/{}-6-768-b32-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #distilbert-b64-L6
    label_accuracy_6_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L6-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #8-768-distilbert
    label_accuracy_8_768 = torch.tensor([get_avg_results('reports/{}-8-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #8-b128-distilbert
    label_accuracy_8_b128 = torch.tensor([get_avg_results('reports/{}-8-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #8-768-s9273-distilbert
    label_accuracy_8_768_s9273 = torch.tensor([get_avg_results('reports/{}-8-768-en-s9273-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #distilbert-b64-L8
    label_accuracy_8_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L8-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #10-768-distilbert
    label_accuracy_10_768 = torch.tensor([get_avg_results('reports/{}-10-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,11)])

    #10-b128-distilbert
    label_accuracy_10_b128 = torch.tensor([get_avg_results('reports/{}-10-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,11)])

    #distilbert-b64-L10
    label_accuracy_10_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L10-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,11)])

    #12-768-distilbert
    label_accuracy_12_768 = torch.tensor([get_avg_results('reports/{}-12-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #12-b128-distilbert
    label_accuracy_12_b128 = torch.tensor([get_avg_results('reports/{}-12-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #distilbert-b64-L12
    #label_accuracy_12_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L12-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #print(label_accuracy)

    #12 layer
    x_range1 = list(range(1,13))
    #3 layer
    x_range2 = np.linspace(1,12,3)
    #6 layer
    x_range3 = np.linspace(1,12,6)
    #8 layer
    x_range4 = np.linspace(1,12,8)
    #10 layer
    x_range5 = np.linspace(1,12,10)

    plt.plot(x_range2, label_accuracy_3_768, label=r'3-768-distilbert', color='red')
    plt.plot(x_range3, label_accuracy_6_768, label=r'6-768-distilbert', color='green')
    plt.plot(x_range3, label_accuracy_6_768_b32, label=r'6-768-b32-distilbert', color='blue')
    plt.plot(x_range3, label_accuracy_6_b128, label=r'6-b128-distilbert', color='yellow')
    plt.plot(x_range3, label_accuracy_6_b64, label=r'distilbert-b64-L6', color='orange')
    plt.plot(x_range4, label_accuracy_8_768, label=r'8-768-distilbert', color='purple')
    plt.plot(x_range4, label_accuracy_8_b128, label=r'8-b128-distilbert', color='brown')
    plt.plot(x_range4, label_accuracy_8_b64, label=r'distilbert-b64-L8', color='hotpink')
    plt.plot(x_range4, label_accuracy_8_768_s9273, label=r'8-768-s9273-distilbert', color='teal')
    plt.plot(x_range5, label_accuracy_10_768, label=r'10-768-distilbert', color='maroon')
    plt.plot(x_range5, label_accuracy_10_b128, label=r'10-b128-distilbert', color='cyan')
    plt.plot(x_range5, label_accuracy_10_b64, label=r'distilbert-b64-L10', color='black')
    plt.plot(x_range1, label_accuracy_12_768, label=r'12-768-distilbert', color='grey')
    plt.plot(x_range1, label_accuracy_12_b128, label=r'12-b128-distilbert', color='lime')
    #plt.plot(x_range5, label_accuracy_12_b64, label=r'distilbert-b64-L12', color='olive')



    plt.title('Accuracy of {} for variants of bert scaled for 12 layers'.format(task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize = "small")
    plt.tight_layout()

    plt.savefig('graphs/Accuracies/Accuracies for {} experiments of bert scaled to 12 layers.png'.format(task), dpi=200)

    plt.clf()

In [80]:
plot_accuracy_experiments('dep_rel')
plot_accuracy_experiments('named_entities')
plot_accuracy_experiments('ptb_pos')
plot_accuracy_experiments('upos')
plot_accuracy_experiments('word_sense')
plot_accuracy_experiments('sst2')

In [82]:
def plot_conditional_experiments(task):
    fig, ax = plt.subplots(figsize=(10,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""
    #3-768-distilbert
    twolayer_3_768 = torch.tensor([get_avg_results('reports/{}-3-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,4)])
    embedded7 = get_avg_results('reports/{}-3-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_3_768 = embedded7 - twolayer_3_768

    #6-768-distilbert
    twolayer_6_768 = torch.tensor([get_avg_results('reports/{}-6-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded4 = get_avg_results('reports/{}-6-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_6_768 = embedded4 - twolayer_6_768

    #6-b128-distilbert
    twolayer_6_b128 = torch.tensor([get_avg_results('reports/{}-6-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded5 = get_avg_results('reports/{}-6-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional5_6_b128 = embedded5 - twolayer_6_b128

    #6-768-b32-distilbert
    twolayer_6_b32 = torch.tensor([get_avg_results('reports/{}-6-768-b32-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded6 = get_avg_results('reports/{}-6-768-b32-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_6_b32 = embedded6 - twolayer_6_b32

    #distilbert-b64-L6
    twolayer_6_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L6-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embeddedb64 = get_avg_results('reports/{}-distilbert-b64-L6-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_6_b64 = embeddedb64 - twolayer_6_b64

    #8-768-distilbert
    twolayer_8_768 = torch.tensor([get_avg_results('reports/{}-8-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded8 = get_avg_results('reports/{}-8-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_768 = embedded8 - twolayer_8_768

    #8-b128-distilbert
    twolayer_8_b128 = torch.tensor([get_avg_results('reports/{}-8-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded8b128 = get_avg_results('reports/{}-8-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_b128 = embedded8b128 - twolayer_8_b128

    #8-768-s9273-distilbert
    twolayer_8_768_S9273 = torch.tensor([get_avg_results('reports/{}-8-768-en-s9273-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded9 = get_avg_results('reports/{}-8-768-en-s9273-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_s9273 = embedded9 - twolayer_8_768_S9273

    #distilbert-b64-L8
    twolayer_8_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L8-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded8b64 = get_avg_results('reports/{}-distilbert-b64-L8-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_b64 = embedded8b64 - twolayer_8_b64

    #10-768-distilbert
    twolayer_10_768 = torch.tensor([get_avg_results('reports/{}-10-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,11)])
    embedded10 = get_avg_results('reports/{}-10-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_10_768 = embedded10 - twolayer_10_768

    #10-b128-distilbert
    twolayer_10_b128 = torch.tensor([get_avg_results('reports/{}-10-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,11)])
    embedded11 = get_avg_results('reports/{}-10-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_10_b128 = embedded11 - twolayer_10_b128

    #distilbert-b64-L10
    twolayer_10_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L10-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,11)])
    embedded10b64 = get_avg_results('reports/{}-distilbert-b64-L10-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_10_b64 = embedded10b64 - twolayer_10_b64

    #12-768-distilbert
    twolayer_12_768 = torch.tensor([get_avg_results('reports/{}-12-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded12 = get_avg_results('reports/{}-12-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_12_768 = embedded12 - twolayer_12_768

    #12-b128-distilbert
    twolayer_12_b128 = torch.tensor([get_avg_results('reports/{}-12-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded12b128 = get_avg_results('reports/{}-12-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_12_b128 = embedded12b128 - twolayer_12_b128

    #distilbert-b64-L12
    # twolayer_12_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L12-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    # embedded12b64 = get_avg_results('reports/{}-distilbert-b64-L12-layer0.yaml.results/dev.v_entropy'.format(task))
    # conditional_12_b64 = embedded12b64 - twolayer_12_b64

    #print(label_accuracy)

    #12 layer
    x_range1 = list(range(1,13))
    #3 layer
    x_range2 = np.linspace(1,12,3)
    #6 layer
    x_range3 = np.linspace(1,12,6)
    #8 layer
    x_range4 = np.linspace(1,12,8)
    #10 layer
    x_range5 = np.linspace(1,12,10)

    plt.plot(x_range2, conditional_3_768, label=r'3-768-distilbert', color='red')
    plt.plot(x_range3, conditional_6_768, label=r'6-768-distilbert', color='green')
    plt.plot(x_range3, conditional_6_b32, label=r'6-768-b32-distilbert', color='blue')
    plt.plot(x_range3, conditional5_6_b128, label=r'6-b128-distilbert', color='yellow')
    plt.plot(x_range3, conditional_6_b64, label=r'distilbert-b64-L6', color='orange')
    plt.plot(x_range4, conditional_8_768, label=r'8-768-distilbert', color='purple')
    plt.plot(x_range4, conditional_8_b128, label=r'8-b128-distilbert', color='brown')
    plt.plot(x_range4, conditional_8_b64, label=r'distilbert-b64-L8', color='hotpink')
    plt.plot(x_range4, conditional_8_s9273, label=r'8-768-s9273-distilbert', color='teal')
    plt.plot(x_range5, conditional_10_768, label=r'10-768-distilbert', color='maroon')
    plt.plot(x_range5, conditional_10_b128, label=r'10-b128-distilbert', color='cyan')
    plt.plot(x_range5, conditional_10_b64, label=r'distilbert-b64-L10', color='black')
    plt.plot(x_range1, conditional_12_768, label=r'12-768-distilbert', color='grey')
    plt.plot(x_range1, conditional_12_b128, label=r'12-b128-distilbert', color='lime')
    #plt.plot(x_range1, conditional_12_b64, label=r'distilbert-b64-L12', color='olive')

    plt.title('V-Entropy of {} for variants of bert scaled for 12 layers'.format(task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize = "small")
    plt.tight_layout()

    plt.savefig('graphs/V-Entropy/V-Entropy for {} experiments of bert scaled to 12 layers.png'.format(task), dpi=200)

    plt.clf()

In [83]:
plot_conditional_experiments('dep_rel')
plot_conditional_experiments('named_entities')
plot_conditional_experiments('ptb_pos')
plot_conditional_experiments('upos')
plot_conditional_experiments('word_sense')
plot_conditional_experiments('sst2')

In [86]:
def plot_accuracy_combined(task):
    fig, ax = plt.subplots(figsize=(10,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""
    #bert768
    label_accuracy_bert768 = torch.tensor([get_avg_results('reports/{}-bert768-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #bert-l2-prune30
    label_accuracy_bert_l2_prune30 = torch.tensor([get_avg_results('reports/{}-bert-l2-prune30-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #distilbert-base-uncased
    label_accuracy_distilbert_base_uncased = torch.tensor([get_avg_results('reports/{}-distilbert-base-uncased-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #MiniLMv2-L6-H768-distilled-from-BERT-Base
    label_accuracy_minilmv2 = torch.tensor([get_avg_results('reports/{}-MiniLMv2-L6-H768-distilled-from-BERT-Base-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #3-768-distilbert
    label_accuracy_3_768 = torch.tensor([get_avg_results('reports/{}-3-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,4)])

    #6-768-distilbert
    label_accuracy_6_768 = torch.tensor([get_avg_results('reports/{}-6-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #6-b128-distilbert
    label_accuracy_6_b128 = torch.tensor([get_avg_results('reports/{}-6-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #6-768-b32-distilbert
    label_accuracy_6_768_b32 = torch.tensor([get_avg_results('reports/{}-6-768-b32-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #distilbert-b64-L6
    label_accuracy_6_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L6-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #8-768-distilbert
    label_accuracy_8_768 = torch.tensor([get_avg_results('reports/{}-8-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #8-b128-distilbert
    label_accuracy_8_b128 = torch.tensor([get_avg_results('reports/{}-8-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #distilbert-b64-L8
    label_accuracy_8_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L8-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #8-768-s9273-distilbert
    label_accuracy_8_768_s9273 = torch.tensor([get_avg_results('reports/{}-8-768-en-s9273-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #10-768-distilbert
    label_accuracy_10_768 = torch.tensor([get_avg_results('reports/{}-10-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,11)])

    #10-b128-distilbert
    label_accuracy_10_b128 = torch.tensor([get_avg_results('reports/{}-10-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,11)])

    #distilbert-b64-L10
    label_accuracy_10_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L10-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,11)])

    #12-768-distilbert
    label_accuracy_12_768 = torch.tensor([get_avg_results('reports/{}-12-768-en-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #12-b128-distilbert
    label_accuracy_12_b128 = torch.tensor([get_avg_results('reports/{}-12-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #distilbert-b64-L12
    #label_accuracy_12_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L12-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #print(label_accuracy)

    #12 layer
    x_range1 = list(range(1,13))
    #3 layer
    x_range2 = np.linspace(1,12,3)
    #6 layer
    x_range3 = np.linspace(1,12,6)
    #8 layer
    x_range4 = np.linspace(1,12,8)
    #10 layer
    x_range5 = np.linspace(1,12,10)

    #plt.plot(x_range1, label_accuracy_bert768, label=r'bert_768')
    #plt.plot(x_range3, label_accuracy_distilbert_base_uncased, label=r'distilbert-base-uncased')
    #plt.plot(x_range3, label_accuracy_minilmv2, label=r'MiniLMv2')
    #plt.plot(x_range1, label_accuracy_bert_l2_prune30, label=r'bert-l2-prune30')
    plt.plot(x_range2, label_accuracy_3_768, label=r'3-768-distilbert')
    plt.plot(x_range3, label_accuracy_6_768, label=r'6-768-distilbert')
    plt.plot(x_range3, label_accuracy_6_768_b32, label=r'6-768-b32-distilbert')
    plt.plot(x_range3, label_accuracy_6_b128, label=r'6-b128-distilbert')
    plt.plot(x_range3, label_accuracy_6_b64, label=r'distilbert-b64-L6')
    plt.plot(x_range4, label_accuracy_8_768, label=r'8-768-distilbert')
    plt.plot(x_range4, label_accuracy_8_b128, label=r'8-b128-distilbert')
    plt.plot(x_range4, label_accuracy_8_b64, label=r'distilbert-b64-L8')
    plt.plot(x_range4, label_accuracy_8_768_s9273, label=r'8-768-s9273-distilbert')
    plt.plot(x_range5, label_accuracy_10_768, label=r'10-768-distilbert')
    plt.plot(x_range5, label_accuracy_10_b128, label=r'10-b128-distilbert')
    plt.plot(x_range5, label_accuracy_10_b64, label=r'distilbert-b64-L10')
    plt.plot(x_range1, label_accuracy_12_768, label=r'12-768-distilbert')
    plt.plot(x_range1, label_accuracy_12_b128, label=r'12-b128-distilbert')
    #plt.plot(x_range1, label_accuracy_12_b64, label=r'distilbert-b64-L12')

    plt.title('Accuracy of {} for variants of bert scaled for 12 layers'.format(task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize = "small")
    plt.tight_layout()

    #plt.savefig('graphs/Accuracies/Accuracies for {} bert768 variants scaled to 12 layers.png'.format(task), dpi=200)
    plt.savefig('graphs/Accuracies/Accuracies for {} wo bert768 scaled to 12 layers.png'.format(task), dpi=200)

    plt.clf()

In [87]:
plot_accuracy_combined('dep_rel')
plot_accuracy_combined('named_entities')
plot_accuracy_combined('ptb_pos')
plot_accuracy_combined('upos')
plot_accuracy_combined('word_sense')
plot_accuracy_combined('sst2')

In [90]:
def plot_conditional_combined(task):
    fig, ax = plt.subplots(figsize=(10,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""
    #bert768
    twolayer_bert768 = torch.tensor([get_avg_results('reports/{}-bert768-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded1 = get_avg_results('reports/{}-bert768-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_bert768 = embedded1 - twolayer_bert768

    #bert-l2-prune30
    twolayer_bertl2 = torch.tensor([get_avg_results('reports/{}-bert-l2-prune30-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded2 = get_avg_results('reports/{}-bert-l2-prune30-layer0.yaml.results/dev.v_entropy'.format(task))
    conditiona_l2 = embedded2 - twolayer_bertl2

    #distilbert-base-uncased
    twolayer_distil = torch.tensor([get_avg_results('reports/{}-distilbert-base-uncased-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded3 = get_avg_results('reports/{}-distilbert-base-uncased-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_distil = embedded3 - twolayer_distil

    #MiniLMv2-L6-H768-distilled-from-BERT-Base
    twolayer_minilmv2 = torch.tensor([get_avg_results('reports/{}-MiniLMv2-L6-H768-distilled-from-BERT-Base-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded13 = get_avg_results('reports/{}-MiniLMv2-L6-H768-distilled-from-BERT-Base-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_minilmv2 = embedded13 - twolayer_minilmv2

    #3-768-distilbert
    twolayer_3_768 = torch.tensor([get_avg_results('reports/{}-3-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,4)])
    embedded7 = get_avg_results('reports/{}-3-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_3_768 = embedded7 - twolayer_3_768

    #6-768-distilbert
    twolayer_6_768 = torch.tensor([get_avg_results('reports/{}-6-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded4 = get_avg_results('reports/{}-6-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_6_768 = embedded4 - twolayer_6_768

    #6-b128-distilbert
    twolayer_6_b128 = torch.tensor([get_avg_results('reports/{}-6-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded5 = get_avg_results('reports/{}-6-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional5_6_b128 = embedded5 - twolayer_6_b128

    #6-768-b32-distilbert
    twolayer_6_b32 = torch.tensor([get_avg_results('reports/{}-6-768-b32-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded6 = get_avg_results('reports/{}-6-768-b32-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_6_b32 = embedded6 - twolayer_6_b32

    #distilbert-b64-L6
    twolayer_6_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L6-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embeddedb64 = get_avg_results('reports/{}-distilbert-b64-L6-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_6_b64 = embeddedb64 - twolayer_6_b64

    #8-768-distilbert
    twolayer_8_768 = torch.tensor([get_avg_results('reports/{}-8-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded8 = get_avg_results('reports/{}-8-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_768 = embedded8 - twolayer_8_768

    #8-b128-distilbert
    twolayer_8_b128 = torch.tensor([get_avg_results('reports/{}-8-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded8b128 = get_avg_results('reports/{}-8-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_b128 = embedded8b128 - twolayer_8_b128

    #8-768-s9273-distilbert
    twolayer_8_768_S9273 = torch.tensor([get_avg_results('reports/{}-8-768-en-s9273-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded9 = get_avg_results('reports/{}-8-768-en-s9273-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_s9273 = embedded9 - twolayer_8_768_S9273

    #distilbert-b64-L8
    twolayer_8_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L8-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded8b64 = get_avg_results('reports/{}-distilbert-b64-L8-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_b64 = embedded8b64 - twolayer_8_b64

    #10-768-distilbert
    twolayer_10_768 = torch.tensor([get_avg_results('reports/{}-10-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,11)])
    embedded10 = get_avg_results('reports/{}-10-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_10_768 = embedded10 - twolayer_10_768

    #10-b128-distilbert
    twolayer_10_b128 = torch.tensor([get_avg_results('reports/{}-10-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,11)])
    embedded11 = get_avg_results('reports/{}-10-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_10_b128 = embedded11 - twolayer_10_b128

    #distilbert-b64-L10
    twolayer_10_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L10-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,11)])
    embedded10b64 = get_avg_results('reports/{}-distilbert-b64-L10-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_10_b64 = embedded10b64 - twolayer_10_b64

    #12-768-distilbert
    twolayer_12_768 = torch.tensor([get_avg_results('reports/{}-12-768-en-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded12 = get_avg_results('reports/{}-12-768-en-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_12_768 = embedded12 - twolayer_12_768

    #12-b128-distilbert
    twolayer_12_b128 = torch.tensor([get_avg_results('reports/{}-12-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded12b128 = get_avg_results('reports/{}-12-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_12_b128 = embedded12b128 - twolayer_12_b128

    #distilbert-b64-L12
    # twolayer_12_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L12-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    # embedded12b64 = get_avg_results('reports/{}-distilbert-b64-L12-layer0.yaml.results/dev.v_entropy'.format(task))
    # conditional_12_b64 = embedded12b64 - twolayer_12_b64

    

    #print(label_accuracy)

    #12 layer
    x_range1 = list(range(1,13))
    #3 layer
    x_range2 = np.linspace(1,12,3)
    #6 layer
    x_range3 = np.linspace(1,12,6)
    #8 layer
    x_range4 = np.linspace(1,12,8)
    #10 layer
    x_range5 = np.linspace(1,12,10)

    #plt.plot(x_range1, conditional_bert768, label=r'bert_768')
    #plt.plot(x_range3, conditional_distil, label=r'distilbert-base-uncased')
    #plt.plot(x_range3, conditional_minilmv2, label=r'MiniLMv2')
    #plt.plot(x_range1, conditiona_l2, label=r'bert-l2-prune30')
    plt.plot(x_range2, conditional_3_768, label=r'3-768-distilbert')
    plt.plot(x_range3, conditional_6_768, label=r'6-768-distilbert')
    plt.plot(x_range3, conditional_6_b32, label=r'6-768-b32-distilbert')
    plt.plot(x_range3, conditional5_6_b128, label=r'6-b128-distilbert')
    plt.plot(x_range3, conditional_6_b64, label=r'distilbert-b64-L6')
    plt.plot(x_range4, conditional_8_768, label=r'8-768-distilbert')
    plt.plot(x_range4, conditional_8_b128, label=r'8-b128-distilbert')
    plt.plot(x_range4, conditional_8_b64, label=r'distilbert-b64-L8')
    plt.plot(x_range4, conditional_8_s9273, label=r'8-768-s9273-distilbert')
    plt.plot(x_range5, conditional_10_768, label=r'10-768-distilbert')
    plt.plot(x_range5, conditional_10_b128, label=r'10-b128-distilbert')
    plt.plot(x_range5, conditional_10_b64, label=r'distilbert-b64-L10')
    plt.plot(x_range1, conditional_12_768, label=r'12-768-distilbert')
    plt.plot(x_range1, conditional_12_b128, label=r'12-b128-distilbert')
    #plt.plot(x_range1, conditional_12_b64, label=r'distilbert-b64-L12')

    plt.title('V-Entropy of {} for variants of bert scaled for 12 layers'.format(task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize = "small")
    plt.tight_layout()

    #plt.savefig('graphs/V-Entropy/V-Entropy for {} bert768 variants scaled to 12 layers.png'.format(task), dpi=200)
    plt.savefig('graphs/V-Entropy/V-Entropy for {} wo bert768 scaled to 12 layers.png'.format(task), dpi=200)

    plt.clf()

In [91]:
plot_conditional_combined('dep_rel')
plot_conditional_combined('named_entities')
plot_conditional_combined('ptb_pos')
plot_conditional_combined('upos')
plot_conditional_combined('word_sense')
plot_conditional_combined('sst2')

In [26]:
def plot_b64_vs_b128_accuracy(task):
    fig, ax = plt.subplots(figsize=(10,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""

    #6-b128-distilbert
    label_accuracy_6_b128 = torch.tensor([get_avg_results('reports/{}-6-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #8-b128-distilbert
    label_accuracy_8_b128 = torch.tensor([get_avg_results('reports/{}-8-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #10-b128-distilbert
    label_accuracy_10_b128 = torch.tensor([get_avg_results('reports/{}-10-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,11)])

    #12-b128-distilbert
    label_accuracy_12_b128 = torch.tensor([get_avg_results('reports/{}-12-b128-distilbert-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])

    #distilbert-b64-L6
    label_accuracy_6_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L6-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,7)])

    #distilbert-b64-L8
    label_accuracy_8_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L8-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,9)])

    #distilbert-b64-L10
    label_accuracy_10_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L10-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,11)])

    #distilbert-b64-L12
    #label_accuracy_12_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L12-layer{}.yaml.results/dev.label_acc'.format(task,i)) for i in range(1,13)])


    #print(label_accuracy)

    #12 layer
    x_range1 = list(range(1,13))
    #3 layer
    x_range2 = np.linspace(1,12,3)
    #6 layer
    x_range3 = np.linspace(1,12,6)
    #8 layer
    x_range4 = np.linspace(1,12,8)
    #10 layer
    x_range5 = np.linspace(1,12,10)

    plt.plot(x_range3, label_accuracy_6_b128, label=r'6-b128-distilbert')
    plt.plot(x_range4, label_accuracy_8_b128, label=r'8-b128-distilbert')
    plt.plot(x_range5, label_accuracy_10_b128, label=r'10-b128-distilbert')
    plt.plot(x_range1, label_accuracy_12_b128, label=r'12-b128-distilbert')
    plt.plot(x_range3, label_accuracy_6_b64, label=r'distilbert-b64-L6')
    plt.plot(x_range4, label_accuracy_8_b64, label=r'distilbert-b64-L8')
    plt.plot(x_range5, label_accuracy_10_b64, label=r'distilbert-b64-L10')
    #plt.plot(x_range1, label_accuracy_12_b64, label=r'distilbert-b64-L12')

    plt.title('Accuracy of {} for batch size 64 vs 128 of bert scaled for 12 layers'.format(task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize = "small")
    plt.tight_layout()

    plt.savefig('graphs/Accuracies/Accuracies for {} b64 vs b128 scaled to 12 layers.png'.format(task), dpi=200)

    plt.clf()

In [ ]:
plot_b64_vs_b128_accuracy('dep_rel')
plot_b64_vs_b128_accuracy('named_entities')
plot_b64_vs_b128_accuracy('ptb_pos')
plot_b64_vs_b128_accuracy('upos')
plot_b64_vs_b128_accuracy('word_sense')
plot_b64_vs_b128_accuracy('sst2')

In [28]:
def plot_b64_vs_b128_conditional(task):
    fig, ax = plt.subplots(figsize=(10,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,13)))
    """Makes a plot comparing layerwise bits of info"""
    #6-b128-distilbert
    twolayer_6_b128 = torch.tensor([get_avg_results('reports/{}-6-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded1 = get_avg_results('reports/{}-6-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_6_b128 = embedded1 - twolayer_6_b128

    #8-b128-distilbert
    twolayer_8_b128 = torch.tensor([get_avg_results('reports/{}-8-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded2 = get_avg_results('reports/{}-8-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_b128 = embedded2 - twolayer_8_b128

    #10-b128-distilbert
    twolayer_10_b128 = torch.tensor([get_avg_results('reports/{}-10-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,11)])
    embedded3 = get_avg_results('reports/{}-10-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_10_b128 = embedded3 - twolayer_10_b128

    #12-b128-distilbert
    twolayer_12_b128 = torch.tensor([get_avg_results('reports/{}-12-b128-distilbert-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    embedded4 = get_avg_results('reports/{}-12-b128-distilbert-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_12_b128 = embedded4 - twolayer_12_b128

    #distilbert-b64-L6
    twolayer_6_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L6-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,7)])
    embedded5 = get_avg_results('reports/{}-distilbert-b64-L6-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_6_b64 = embedded5 - twolayer_6_b64

    #distilbert-b64-L8
    twolayer_8_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L8-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,9)])
    embedded6 = get_avg_results('reports/{}-distilbert-b64-L8-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_8_b64 = embedded6 - twolayer_8_b64

    #distilbert-b64-L10
    twolayer_10_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L10-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,11)])
    embedded7 = get_avg_results('reports/{}-distilbert-b64-L10-layer0.yaml.results/dev.v_entropy'.format(task))
    conditional_10_b64 = embedded7 - twolayer_10_b64

    #distilbert-b64-L12
    #twolayer_12_b64 = torch.tensor([get_avg_results('reports/{}-distilbert-b64-L10-layer{}-0.yaml.results/dev.v_entropy'.format(task,i)) for i in range(1,13)])
    #embedded8 = get_avg_results('reports/{}-distilbert-b64-L10-layer0.yaml.results/dev.v_entropy'.format(task))
    #conditional_12_b64 = embedded8 - twolayer_12_b64

    #12 layer
    x_range1 = list(range(1,13))
    #3 layer
    x_range2 = np.linspace(1,12,3)
    #6 layer
    x_range3 = np.linspace(1,12,6)
    #8 layer
    x_range4 = np.linspace(1,12,8)
    #10 layer
    x_range5 = np.linspace(1,12,10)


    plt.plot(x_range3, conditional_6_b128, label=r'6-b128-distilbert')
    plt.plot(x_range4, conditional_8_b128, label=r'8-b128-distilbert')
    plt.plot(x_range5, conditional_10_b128, label=r'10-b128-distilbert')
    plt.plot(x_range1, conditional_12_b128, label=r'12-b128-distilbert')
    plt.plot(x_range3, conditional_6_b64, label=r'6-b64-distilbert')
    plt.plot(x_range4, conditional_8_b64, label=r'8-b64-distilbert')
    plt.plot(x_range5, conditional_10_b64, label=r'10-b64-distilbert')
    #plt.plot(x_range1, conditional_12_b128, label=r'12-b64-distilbert')

    plt.title('V-Entropy of {} for batch size 64 vs 128 of bert scaled for 12 layers'.format(task), fontsize = 12)

    plt.ylabel('Accuracy', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    #plt.legend()
    plt.legend(bbox_to_anchor=(1.05, 1.0), loc='upper left', fontsize = "small")
    plt.tight_layout()

    plt.savefig('graphs/V-Entropy/V-Entropy for {} b64 vs b128 scaled to 12 layers.png'.format(task), dpi=200)

    plt.clf()

In [29]:
plot_b64_vs_b128_conditional('dep_rel')
plot_b64_vs_b128_conditional('named_entities')
plot_b64_vs_b128_conditional('ptb_pos')
plot_b64_vs_b128_conditional('upos')
plot_b64_vs_b128_conditional('word_sense')
plot_b64_vs_b128_conditional('sst2')

In [ ]:
#tests
def plot_layerwise_adjacent(task, model):
    fig, ax = plt.subplots(figsize=(6,4.5))
    ax.tick_params(axis='both', which='major', labelsize=15)
    ax.tick_params(axis='both', which='minor', labelsize=15)
    #change depending on layers
    ax.set_xticks(list(range(1,7)))
    """Makes a plot comparing layerwise bits of info"""
    #change range depending on layers of model
    task_768_singlelayer_ventropy = torch.tensor([get_avg_results('reports/{}-{}-layer{}.yaml.results/dev.v_entropy'.format(task,model,i)) for i in range(1,7)])
    task_768_twolayer_ventropy = torch.tensor([get_avg_results('reports/{}-{}-layer{}-{}.yaml.results/dev.v_entropy'.format(task,model,i-1,i)) for i in range(1,7)])
    task_768_embedding_layer_ventropy = torch.tensor([get_avg_results('reports/{}-{}-layer{}.yaml.results/dev.v_entropy'.format(task,model,i-1)) for i in range(1,7)])

    #print(task_768_singlelayer_ventropy)
    #print(task_768_twolayer_ventropy)
    #print(task_768_embedding_layer_ventropy)

    #12 layer
    #x_range = list(range(1,13))

    #6 layer
    x_range = list(range(1,7))

    #3 layer
    #x_range = list(range(1,4))

    #8 layer
    #x_range = list(range(1,9))

    #10 layer
    #x_range = list(range(1,11))

    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_singlelayer_ventropy, label=r'baselined')
    plt.plot(x_range, task_768_embedding_layer_ventropy-task_768_twolayer_ventropy, label=r'conditional')

    #plot title
    plt.title('{} {}'.format(model,task), fontsize = 12)

    plt.ylabel('Bits', fontsize = 12)
    plt.xlabel('Model layer', fontsize = 12)
    plt.legend()
    plt.tight_layout()

    #save figure in folder
    plt.savefig('graphs/{0}/{0} {1} test.png'.format(model,task), dpi=200)

    plt.clf()

In [ ]:
plot_layerwise_adjacent('dep_rel','MiniLMv2-L6-H768-distilled-from-BERT-Base')
